In [ ]:
!pip install faiss_cpu

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BertTokenizer, BertModel
import torch
import numpy as np
import faiss

# GPT-2モデルとトークナイザーのロード
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

# BERTモデルとトークナイザーのロード
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_sentence_vector(sentence, model, tokenizer):
    """文のベクトル化を行う関数"""
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=128)
    outputs = model(**inputs)
    sentence_vector = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return sentence_vector.reshape(-1)

def search_closest_question(query, index, qa_pairs, model, tokenizer):
    """FAISSインデックスを使用して、質問に最も近いQAペアを検索する関数"""
    query_vector = get_sentence_vector(query, model, tokenizer)
    D, I = index.search(np.array([query_vector]), k=1)
    closest_pair = qa_pairs[I[0][0]]
    return closest_pair

# 質問と回答のペアのリスト
qa_pairs = [
    "質問: 「注文した商品をどのように追跡しますか？」 回答: 「注文番号と電子メールアドレスを使用して、当社のウェブサイト上で注文を追跡できます。」",
    "質問: 「製品の返品方法は？」 回答: 「製品は購入後30日以内に返品することができます。返品するには、オンラインフォームを記入してください。」",
    "質問: 「支払い方法にはどのようなものがありますか？」 回答: 「クレジットカード、デビットカード、PayPal、銀行振込を受け付けています。」",
    "質問: 「配送料金はいくらですか？」 回答: 「配送料金はお住まいの地域によって異なります。詳細はお支払いページをご覧ください。」",
    "質問: 「商品はどれくらいの日数で届きますか？」 回答: 「通常、商品の配送には3〜5営業日かかります。」",
    "質問: 「カスタマーサポートに連絡する方法は？」 回答: 「カスタマーサポートには電話、メール、お問い合わせフォームで連絡できます。詳細はお問い合わせページをご覧ください。」",
    "質問: 「商品が壊れて届いた場合、どうすれば良いですか？」 回答: 「壊れた商品の場合、カスタマーサポートに連絡し、交換または返金の手続きを行ってください。」",
    "質問: 「商品の在庫状況はどこで確認できますか？」 回答: 「商品の在庫状況は製品ページで確認できます。在庫がある場合、数量が表示されます。」",
    "質問: 「注文をキャンセルする方法は？」 回答: 「注文をキャンセルするには、注文番号を入力し、カスタマーサポートに連絡してください。」",
    "質問: 「返品送料は誰が負担しますか？」 回答: 「返品送料は、壊れた商品または誤った商品が送られた場合を除き、お客様の負担となります。」",
    "質問: 「商品の保証期間はどのくらいですか？」 回答: 「商品の保証期間は通常1年です。詳細は製品の保証ポリシーをご確認ください。」"
]

# 各ペアをベクトル化
vectors = [get_sentence_vector(pair, bert_model, bert_tokenizer) for pair in qa_pairs]

# FAISSを使用してベクトルデータベースを作成
dim = vectors[0].shape[0]
index = faiss.IndexFlatL2(dim)
index.add(np.array(vectors))  # ベクトルを追加

def generate_answer_with_gpt(document, query, tokenizer, model):
    # パディングトークンの設定
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

    """文書と質問に基づいてGPT-2で回答を生成する関数"""
    combined_input = query + " " + document
    inputs = tokenizer.encode_plus(combined_input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # GPT-2モデルで回答を生成
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=300)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# ユーザーからの質問
user_question = "支払い方法は何がありますか？"

# 関連するQAペアを検索
closest_pair = search_closest_question(user_question, index, qa_pairs, bert_model, bert_tokenizer)

# # GPT-2を使用して回答を生成
# answer = generate_answer_with_gpt(closest_pair, user_question, gpt_tokenizer, gpt_model)
# print(answer)

# GPT-2モデルを使用して生成された回答から「回答:」の後の部分を抽出
def extract_answer(text):
    parts = text.split("回答: ")
    if len(parts) > 1:
        return parts[1].split("」")[0]
    else:
        return "回答が見つかりませんでした。"

# 回答を生成
generated_answer = generate_answer_with_gpt(closest_pair, user_question, gpt_tokenizer, gpt_model)

# 回答から必要な部分を抽出
final_answer = extract_answer(generated_answer)
print(final_answer)
